In [1]:
import parselmouth as ps
from parselmouth.praat import call as pcall  
import audiolabel # you may need to install this from https://github.com/rsprouse/audiolabel; email me if you get stuck!

This notebook assumes that there are at least the two following audiofiles in the same directory as the notebook. If your files are in another location, or if you want to use different audiofiles, you can point your script to new places.

In [2]:
breathytest = ps.Sound('./21_06_25_TikTok_14_mono.wav') # ps.Sound is the equivalent of opening an audiofile
modaltest = ps.Sound('./21_6_25_mestres_modal.wav')
breathytest # breathytest and modaltest are the Parselmouth analogue to Sound objects in Praat

In [3]:
# equivalent to selecting Analyze periodicity > to harmonicity. You can add the optional variable 'minimum_pitch' 
# to see what that changes if desired. (Uncomment these commands to play around!)
h_breathy = breathytest.to_harmonicity_cc()
# h_breathy = breathytest.to_harmonicity_cc(minimum_pitch=100)
h_modal = modaltest.to_harmonicity_cc()
# h_modal = modaltest.to_harmonicity_cc(minimum_pitch=100)
h_breathy

I went ahead and created TextGrids for the two audio files we were playing around with, with a single interval tier 'vowels', with the labels 1-4, indicating how far from the transition they were. The following lines read these TextGrids in to a "Label Manager" object, which is from the package `audiolabel` which you can get on Ron's github https://github.com/rsprouse/audiolabel

In [4]:
tg_breathy = audiolabel.LabelManager(from_file='./21_06_25_TikTok_14_mono.TextGrid', from_type='praat')
tg_modal = audiolabel.LabelManager(from_file='./21_6_25_mestres_modal.TextGrid', from_type='praat')

LabelManagers are basically lists of intervals or tiers. For interval tiers, each label consists of a start time (`t1`), an end time (`t2`) and a label (`text`). There are several other *really* cool things that `audiolabel` will let us do, and we'll see some of them as we move forward.

In [5]:
tg_breathy.tier('vowels')

IntervalTier([Label( t1=0.0000, t2=10.3060, text='b''' ),Label( t1=10.3060, t2=10.4493, text='b'4'' ),Label( t1=10.4493, t2=10.5410, text='b''' ),Label( t1=10.5410, t2=10.5842, text='b'3'' ),Label( t1=10.5842, t2=10.7007, text='b''' ),Label( t1=10.7007, t2=10.7421, text='b'2'' ),Label( t1=10.7421, t2=10.8882, text='b''' ),Label( t1=10.8882, t2=10.9461, text='b'1'' ),Label( t1=10.9461, t2=19.9310, text='b''' )])

You'll notice that the above cell results in listing *all* of the intervals, but really we're only interested in the labelled tiers. Here's one way you could do that with some slightly fancy list/for action. (Python is particularly good at this kind of thing!)

In [6]:
[v for v in tg_breathy.tier('vowels') if v.text!='']

[Label( t1=10.3060, t2=10.4493, text='b'4'' ),
 Label( t1=10.5410, t2=10.5842, text='b'3'' ),
 Label( t1=10.7007, t2=10.7421, text='b'2'' ),
 Label( t1=10.8882, t2=10.9461, text='b'1'' )]

Now we have all of the pieces we need to get the HNR values from the midpoints of each vowel. The following for loop goes through each of the vowel intervals with a non-empty label (see above), then gets the harmonicity values at midpoint, and prints the label, the time (to 3 decimal places), and the HNR values (to 2 decimal places).

In [7]:
for v in [v for v in tg_breathy.tier('vowels') if v.text!='']:
    harm = h_breathy.get_value((v.t1+v.t2)/2)
    label = v.text
    print('\t'.join([label, "{:.3f}".format((v.t1+v.t2)/2), "{:.2f}".format(harm)]))

4	10.378	15.82
3	10.563	9.45
2	10.721	21.44
1	10.917	-200.00


But maybe that's not quite what we want -- maybe instead, we'd like to be able to get the *mean* harmonicity values from each of the vowels. Unfortunately, Parselmouth does not yet have the "Get mean" function that we found in Praat built in. Luckily though, the author of Parselmouth gave us a way to get around this problem, by using a Parselmouth function called `pcall`. It takes as arguments the Object, the Name of the function in Praat (including any capitalization and ...), and all arguments. 

In [8]:
for v in [v for v in tg_breathy.tier('vowels') if v.text!='']:
    # Referencing the Praat Harmonicity object, we can see that 'Get mean...' takes two arguments, a start time and an end time
    harm = pcall(h_breathy, 'Get mean...', v.t1, v.t2) 
    label = v.text
    print('\t'.join([label, "{:.3f}".format(v.t1), "{:.3f}".format(v.t2), "{:.2f}".format(harm)]))

4	10.306	10.449	8.79
3	10.541	10.584	7.45
2	10.701	10.742	4.35
1	10.888	10.946	nan


And then let's do it again for the modal example

In [9]:
for v in [v for v in tg_modal.tier('vowels') if v.text!='']:
    harm = pcall(h_modal, 'Get mean...', v.t1, v.t2) 
    label = v.text
    print('\t'.join([label, "{:.3f}".format(v.t1), "{:.3f}".format(v.t2), "{:.2f}".format(harm)]))

4	7.731	7.801	14.41
3	7.884	7.970	22.04
2	8.094	8.161	10.70
1	8.280	8.390	7.10


From this small test we can see that Harmonicity does seem to do a good job with respect to our perceptions of breathiness! But we'll need many more observations....